In [43]:
import numpy as np
from skcriteria.preprocessing.scalers import (
            matrix_scale_by_cenit_distance as scale,
        )

from skcriteria.utils import rank
from skcriteria.core import data
from skcriteria import core
MAXS_O_ALIAS = list(core.Objective._MAX_ALIASES.value)
MINS_O_ALIAS = list(core.Objective._MIN_ALIASES.value)
def make(
        *,
        seed=None,
        min_alternatives=256,
        max_alternatives=256,
        min_criteria=50,
        max_criteria=50,
        min_objectives_proportion=0.5,
        nan_proportion=0,
    ):
        # start the random generator
        random = np.random.default_rng(seed=seed)

        # determine the number os alternatives and criteria
        alternatives_number = (
            random.integers(min_alternatives, max_alternatives)
            if min_alternatives != max_alternatives
            else min_alternatives
        )
        criteria_number = (
            random.integers(min_criteria, max_criteria)
            if min_criteria != max_criteria
            else min_criteria
        )

        # create the data matrix with rows = alt and columns = crit
        mtx = random.random((alternatives_number, criteria_number))

        # if we have a nan ratio >0 of nan we have to add them randomly
        # in the matrix
        if nan_proportion:
            nan_number = round(mtx.size * float(nan_proportion))
            nan_positions = random.choice(mtx.size, nan_number, replace=False)
            mtx.ravel()[nan_positions] = np.nan

        # determine the number of minimize objectives based on the proportion
        # of the total number of criteria, and the maximize is the complement
        min_objectives_number = round(
            criteria_number * min_objectives_proportion
        )
        max_objectives_number = round(
            criteria_number * 1.0 - min_objectives_number
        )

        # if the proportion is lt or gt than the total number of criteria
        # we add or remove an objective
        while min_objectives_number + max_objectives_number < criteria_number:
            if random.choice([True, False]):
                max_objectives_number += 1
            else:
                min_objectives_number += 1
        while min_objectives_number + max_objectives_number > criteria_number:
            if random.choice([True, False]):
                max_objectives_number -= 1
            else:
                min_objectives_number -= 1

        # finally we extract the objectives and shuffle the order
        objectives = np.concatenate(
            [
                random.choice(MINS_O_ALIAS, min_objectives_number),
                random.choice(MAXS_O_ALIAS, max_objectives_number),
            ]
        )
        random.shuffle(objectives)

        # the weights
        weights = random.random(criteria_number)

        # create the names of the criteria and the alternatives
        alternatives = [f"A{idx}" for idx in range(alternatives_number)]
        criteria = [f"C{idx}" for idx in range(criteria_number)]

        return mtx, objectives, weights, alternatives, criteria

mtx, objectives, weights, alternatives, criteria = make()

dm = data.mkdm(
        matrix=mtx,
        objectives=objectives,
        weights=weights,
        alternatives=alternatives,
        criteria=criteria,
    )
print(dm)
dict_=dm.to_dict()

matrix = dict_["matrix"]
objectives = dict_["objectives"]
weights = dict_["weights"]

matrix_scaled = scale(matrix, objectives * -1) * weights

# New criteria: Manhattan distance and Chebyshev distance
def ncriteria_to_2criteria(alternative):
    return (np.sum(alternative), np.max(alternative))

# N criteria problem -> 2 criteria problem
distances_matrix = np.apply_along_axis(
    ncriteria_to_2criteria, 1, matrix_scaled
)
# distances_matrix = np.column_stack((
#     np.sum(matrix_scaled), np.max(matrix_scaled))
# ) # Alternative, to be checked
distances_matrix_scaled = scale(distances_matrix, [1, 1])
# We now do weighted sum of our 2 criteria with weights [v, 1-v]
q_k = np.dot(distances_matrix_scaled, [0.5, 0.5])
# Rank them
rank_q_k = rank.rank_values(q_k, reverse=False)

# Check if solution is acceptable
has_rank1_qs = np.where(
    rank_q_k == 1, 1, 0
)  # probably can delete this array
rank1_cnt = np.sum(has_rank1_qs == 1)

# best_q_value = q_k[chosen_qs[0][0]]  # The value of the best q
best_q_value = np.min(q_k)

# DEBUG(chosen_qs,best_q_value, best_qq_value)
dq = 1 / (len(matrix) - 1)
qs_with_acceptable_advantage = np.where(q_k - best_q_value < dq)

# chosen_qs always have acc. adv., therefore same len <=> same qs
has_acceptable_advantage = (
    len(qs_with_acceptable_advantage[0]) == rank1_cnt
)



      C0[▲ 0.796146]  C1[▼ 0.309714]  ...  C48[▲ 0.938330]  C49[▲ 0.057784]
A0          0.700230        0.711267  ...         0.001915         0.030151
A1          0.220536        0.719778  ...         0.232589         0.162975
A2          0.503102        0.449932  ...         0.488931         0.123332
A3          0.383397        0.733205  ...         0.439849         0.894847
A4          0.037307        0.640961  ...         0.111241         0.360982
...              ...             ...  ...              ...              ...
A251        0.005544        0.239145  ...         0.671346         0.422388
A252        0.428952        0.394405  ...         0.412408         0.143422
A253        0.145775        0.681670  ...         0.996673         0.955625
A254        0.503414        0.692740  ...         0.866759         0.241165
A255        0.660705        0.283938  ...         0.279599         0.452701
[256 Alternatives x 50 Criteria]


/home/luigi/scikit-criteria/env/lib/python3.13/site-packages/pandas/io/formats/format.py:1354: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  for val, m in zip(values.ravel(), mask.ravel())


In [44]:
%%timeit
has_any_best_coordinate = np.where(distances_matrix_scaled[:,1] * distances_matrix_scaled[:,0] == 0, 1, 0)
stables_rank1_cnt = np.sum(has_rank1_qs * has_any_best_coordinate == 1)
has_acceptable_stability = rank1_cnt == stables_rank1_cnt


6.22 μs ± 124 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [45]:
%%timeit
chosen_qs = np.where(rank_q_k == 1)  # Possibly many qs with rank 1
bests = np.any(distances_matrix_scaled == 0, axis=1).nonzero()
has_acceptable_stability = np.isin(chosen_qs, bests).all()


25 μs ± 2.76 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [46]:
%%timeit
multiplied_together = np.prod(distances_matrix_scaled, axis=1)
magic = rank_q_k * multiplied_together# todo: check types
zero_cnt = np.count_nonzero(magic == 0)
magic_cnt = np.count_nonzero(multiplied_together == magic)
has_acceptable_stability = zero_cnt == magic_cnt


6.15 μs ± 21.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [47]:
%%timeit
chosen_qs = np.where(rank_q_k == 1)  # Possibly many qs with rank 1
bests = np.any(distances_matrix_scaled == 0, axis=1).nonzero()
has_acceptable_stability = set(chosen_qs[0]).issubset(set(bests[0]))

8.29 μs ± 89.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
